In [56]:
# import json module
import json
import csv
import copy
import pickle
from sklearn.externals import joblib

f = open('ranker.json')
json_data = json.load(f)
f.close()

# ranker 들의 푼 문제들 csv 파일로 저장
with open('ranker.csv', 'w', newline='') as file:
    # type : list -> dictionary
    for ranker in range(0,len(json_data)):
        dic = json.loads(json_data[ranker])
    
        writer = csv.writer(file)
        writer.writerow(dic["solving_problem"])

In [2]:
import numpy as np
import pandas as pd

f = open('ranker.csv', 'r')

lines = csv.reader(f)

# 0번 인덱스는 1000번 문제
# 문제번호 = 인덱스 + 1000
arr = np.zeros((len(json_data),18005))

ranker_num = 0
for line in lines:    
    for idx in range(0, len(line)):
        arr[ranker_num][int(line[idx])-1000] = 1
    ranker_num += 1

In [3]:
'''df = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
df['target'] = diabetes.target
print(df.head())'''

df = pd.DataFrame(arr)
print(df.head())

   0      1      2      3      4      5      6      7      8      9      ...  \
0    1.0    1.0    0.0    1.0    0.0    1.0    0.0    0.0    1.0    0.0  ...   
1    1.0    1.0    1.0    1.0    1.0    1.0    1.0    0.0    1.0    0.0  ...   
2    1.0    1.0    1.0    1.0    0.0    0.0    0.0    0.0    1.0    1.0  ...   
3    1.0    1.0    0.0    1.0    1.0    0.0    0.0    0.0    1.0    1.0  ...   
4    1.0    1.0    1.0    1.0    0.0    0.0    0.0    0.0    1.0    1.0  ...   

   17995  17996  17997  17998  17999  18000  18001  18002  18003  18004  
0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
1    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
2    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
3    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
4    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  

[5 rows x 18005 columns]


In [4]:
tmp_des = df.describe()

In [90]:
tmp_des

,0,1,2,3,4,5,6,7,8,9,...,17995,17996,17997,17998,17999,18000,18001,18002,18003,18004
count,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
mean,0.767000,0.676000,0.10000,0.248000,0.055000,0.039000,0.003000,0.001000,0.534000,0.057000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.422954,0.468234,0.30015,0.432068,0.228094,0.193692,0.054717,0.031623,0.499092,0.231959,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,1.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,1.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# 정답률이 95% 이상이거나 5% 미만 문제는 제거
alive_col = np.zeros(18005)
num = 0
for idx in range(0, len(alive_col)):
    if tmp_des[idx]["mean"] >= 0.95:
        del df[idx]
    elif tmp_des[idx]["mean"] <= 0.05:
        del df[idx]
    else:
        alive_col[num] = idx
        num += 1
    

In [91]:
# 열 제거 후 data set 모습
print(df.head())
# 추천 후보군에 포함되는 문제들(column)
print(alive_col.tolist())

   0      1      2      3      4      8      9      10     11     12     ...  \
0    1.0    1.0    0.0    1.0    0.0    1.0    0.0    0.0    1.0    0.0  ...   
1    1.0    1.0    1.0    1.0    1.0    1.0    0.0    1.0    0.0    0.0  ...   
2    1.0    1.0    1.0    1.0    0.0    1.0    1.0    1.0    0.0    1.0  ...   
3    1.0    1.0    0.0    1.0    1.0    1.0    1.0    1.0    0.0    1.0  ...   
4    1.0    1.0    1.0    1.0    0.0    1.0    1.0    0.0    0.0    0.0  ...   

   10719  10720  10721  10724  10726  10727  12458  13501  13502  14552  
0    1.0    1.0    1.0    0.0    0.0    0.0    0.0    0.0    0.0    1.0  
1    1.0    1.0    1.0    0.0    0.0    0.0    1.0    1.0    0.0    0.0  
2    1.0    1.0    1.0    0.0    0.0    0.0    1.0    0.0    0.0    1.0  
3    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
4    1.0    1.0    1.0    0.0    0.0    0.0    0.0    0.0    0.0    1.0  

[5 rows x 159 columns]
[0.0, 1.0, 2.0, 3.0, 4.0, 8.0, 9.0, 10.0, 11.0, 12.

In [82]:
for idx in range(0, X_train.shape[1]+1):
    print("문제번호", idx+1000)
    X_train = df.copy() # 학습할 dataset 복사
    X_train[alive_col[idx]] = 0 # 학습 문제 값 제거
    Y_train = df[alive_col[idx]].copy() # Y_train 복사
    logreg = LogisticRegression(C=1e2, solver='lbfgs', multi_class='multinomial')
    logreg.fit(X_train, Y_train)
  
    # 모델 저장
    saved_model = pickle.dumps(logreg)
    joblib.dump(logreg, 'model_' + str(int(alive_col[idx]+1000)) +'.pkl')

문제번호 1000
문제번호 1001


C:\Users\Won\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Won\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Won\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


문제번호 1002
문제번호 1003
문제번호 1004


C:\Users\Won\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Won\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


문제번호 1005
문제번호 1006


...

문제번호 1154
문제번호 1155


C:\Users\Won\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Won\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


문제번호 1156
문제번호 1157


C:\Users\Won\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Won\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Won\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


문제번호 1158
문제번호 1159


In [79]:
alive_col.tolist()

[0.0,
 1.0,
 2.0,
 3.0,
 4.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 26.0,
 65.0,
 85.0,
 94.0,
 110.0,
 149.0,
 152.0,
 157.0,
 158.0,
 181.0,
 193.0,
 237.0,
 260.0,
 316.0,
 330.0,
 427.0,
 463.0,
 475.0,
 546.0,
 697.0,
 699.0,
 753.0,
 874.0,
 912.0,
 920.0,
 924.0,
 929.0,
 931.0,
 932.0,
 934.0,
 966.0,
 977.0,
 978.0,
 991.0,
 1156.0,
 1163.0,
 1178.0,
 1193.0,
 1231.0,
 1292.0,
 1293.0,
 1294.0,
 1309.0,
 1438.0,
 1439.0,
 1440.0,
 1441.0,
 1442.0,
 1443.0,
 1444.0,
 1445.0,
 1446.0,
 1455.0,
 1475.0,
 1490.0,
 1557.0,
 1558.0,
 1562.0,
 1576.0,
 1577.0,
 1579.0,
 1581.0,
 1588.0,
 1606.0,
 1609.0,
 1667.0,
 1675.0,
 1739.0,
 1741.0,
 1742.0,
 1743.0,
 1747.0,
 1748.0,
 1750.0,
 1751.0,
 1753.0,
 1775.0,
 1839.0,
 1869.0,
 1884.0,
 1902.0,
 1908.0,
 1920.0,
 1941.0,
 2046.0,
 2052.0,
 3344.0,
 3673.0,
 3948.0,
 3963.0,
 4622.0,
 5064.0,
 6287.0,
 6576.0,
 7393.0,
 7958.0,
 8012.0,
 8095.0,
 8461.0,
 8465.0,
 8498.0,
 9039.0,
 9171.0,
 9172.0,
 9250.0,
 9430.0,
 9718.0,
 9797.0,
 97

In [23]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(C=1e2, solver='lbfgs', multi_class='multinomial')
# 1000번 문제 값 삭제
logreg.fit(X_train, Y_train)

C:\Users\Won\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=100.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [53]:
import pickle
from sklearn.externals import joblib

# 모델 저장
saved_model = pickle.dumps(logreg)
joblib.dump(logreg, 'model_1000' + '.pkl')

['model_1000.pkl']

In [46]:
ranking_1 = df.loc[0].copy()
ranking_1[0] = 0

In [48]:
print(logreg.predict_proba([ranking_1]))

[[7.50825326e-23 1.00000000e+00]]


In [ ]:
print(logreg.predict([[0,0,0,1],[0,1,1,1],[1,1,1,1],[1,1,1,0]]))

In [19]:
print(logreg.score(X_train, Y_train))
print(logreg.score(X_test, Y_test))

1.0
1.0
